<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/OPENAI_AAI_DEMO_LIBRARY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OpenAI Agents SDK: Multi-Agent Handoff

In [ ]:
# Ensure openai-agents is installed for the OpenAI Agents SDK demo
!pip install -q openai-agents
!pip install -q colab-env

In [ ]:
from IPython import get_ipython
from IPython.display import display
import asyncio
import os
import random
import re
import pandas as pd
from agents import Agent, Tool, Runner # Assuming 'agents' library is correctly installed
from sklearn.linear_model import LogisticRegression
from openai import OpenAI, AsyncOpenAI
import colab_env # For Google Colab environment variables


# Initialize AsyncOpenAI client
client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


class FlightEnvironment:
    def __init__(self, origin, destination, weather):
        self.origin = origin
        self.destination = destination
        self.weather = weather

    def get_possible_routes(self):
        routes = {
            ("New York", "London"): [
                {
                    "flight_number": "AC888",
                    "departure_time": "20:00",
                    "arrival_time": "08:00",
                    "price": 800,
                    "airline": "Air Canada",
                    "aircraft_type": "Boeing 787-9",
                },
                {
                    "flight_number": "UA901",
                    "departure_time": "22:00",
                    "arrival_time": "10:00",
                    "price": 900,
                    "airline": "United Airlines",
                    "aircraft_type": "Boeing 777-300ER",
                },
            ]
        }
        return routes.get((self.origin, self.destination), [])

    def get_weather_data(self, location):
        weather_conditions = ["Sunny", "Cloudy", "Rainy", "Overcast", "Snowy"]
        wind_speed = random.randint(5, 20)
        return {
            "weather": [{"description": random.choice(weather_conditions)}],
            "wind": {"speed": wind_speed},
        }

    def update(self):
        print("Updating environment...")
        self.weather = {
            self.origin: self.get_weather_data(self.origin),
            self.destination: self.get_weather_data(self.destination),
        }


class FlightAgent(Agent):
    def __init__(self, environment, turbulence_dataset_path, **kwargs):
        super().__init__(
            name="FlightPlanner",
            model="gpt-4o",
            instructions="You are a flight planning expert. Generate a detailed and safe flight plan based on the provided information.",
            **kwargs,
        )

        self.environment = environment
        self.turbulence_model = LogisticRegression()
        self.turbulence_dataset_path = turbulence_dataset_path
        self.train_turbulence_model()
        self.prompt = None # Initialize prompt to None

    def train_turbulence_model(self):
        try:
            turbulence_df = pd.read_csv(self.turbulence_dataset_path)
            turbulence_df["Turbulence"] = (turbulence_df["REF_k"] > turbulence_df["REF_k"].mean()).astype(int)
            target = turbulence_df["Turbulence"]
            features = turbulence_df[
                [
                    "REF_U_1",
                    "REF_U_2",
                    "REF_U_3",
                    "REF_tau_11",
                    "REF_tau_12",
                    "REF_tau_13",
                    "REF_tau_22",
                    "REF_tau_23",
                    "REF_tau_33",
                ]
            ]
            self.turbulence_model.fit(features, target)
            print("Turbulence model trained successfully!")
        except FileNotFoundError:
            print(f"Error: Turbulence dataset not found at {self.turbulence_dataset_path}. Skipping turbulence model training.")
        except Exception as e:
            print(f"An error occurred during turbulence model training: {e}")

    async def run(self, max_iterations=5):
        """Executes the OODA loop for the FlightAgent."""
        for iteration in range(max_iterations):
            print(f"\n--- OODA Loop Iteration {iteration + 1} ---")
            observations = {
                "origin_weather": self.environment.get_weather_data(self.environment.origin),
                "destination_weather": self.environment.get_weather_data(self.environment.destination),
                "possible_routes": self.environment.get_possible_routes(),
            }
            print("Observations:", observations)
            situation_summary = f"""
            Current Situation:
            - Origin Weather: {observations['origin_weather']['weather'][0]['description']}
            - Destination Weather: {observations['destination_weather']['weather'][0]['description']}
            - Available Routes: {observations['possible_routes']}
            """
            print(situation_summary)

            # Generate flight plan using OpenAIAgent's call_model (Corrected)
            flight_plan = await self.generate_flight_plan(observations)

            print("Flight Plan:", flight_plan)
            print("Simulating flight execution...")
            self.environment.update()

    async def generate_flight_plan(self, observations):
        origin_weather = self.environment.get_weather_data(self.environment.origin)
        destination_weather = self.environment.get_weather_data(self.environment.destination)
        possible_routes = self.environment.get_possible_routes()

        print('\n\n')
        print(f"Weather in {self.environment.origin}: {origin_weather['weather'][0]['description']}")
        print(f"Weather in {self.environment.destination}: {destination_weather['weather'][0]['description']}")
        print("Available flights:")
        print('\n')
        print(f"Origin: {self.environment.origin}")
        print(f"Destination: {self.environment.destination}")
        print('\n')
        for route in possible_routes:
            print(route)

        if not possible_routes:
            return "No flights found."

        """Generates a flight plan using the embedded OpenAI model."""
        prompt = f"""
        Generate a detailed flight plan for a long-haul flight from {self.environment.origin} to {self.environment.destination},
        considering the following:

        ## Recommended Flight and Aircraft Type:
        - Consider these available flights: {observations['possible_routes']}
        - Only consider flights operated by Air Canada (AC) or United Airlines (UA).
        - Based on the available flights, recommend the most suitable flight and provide a justification for your choice.
        - **Clearly state the aircraft type for the recommended flight (e.g., Boeing 777, Airbus A330).**

        ## Route and Waypoints:
        - Determine and include a list of suitable waypoints for the recommended flight, using standard aviation codes (e.g., KJFK, EGLL).
        - Consider typical North Atlantic Tracks (NATs) for optimal routing and fuel efficiency.
        - The origin and destination airport codes are: {self.environment.origin} and {self.environment.destination}.

        ## Fuel Calculations:
        - **Provide a detailed breakdown of fuel requirements for different phases of the flight (e.g., taxi, takeoff, climb, cruise, descent, landing).**
        - Include the total fuel required for the trip.

        ## ETOPS Considerations:
        - State the ETOPS certification for the recommended aircraft.
        - List suitable ETOPS alternate airports.

        ## Risks and Mitigation Strategies:
        - **Provide detailed explanations of potential risks associated with this specific flight and aircraft type, along with specific mitigation strategies.**
        - Consider risks such as crew fatigue, jet lag, clear air turbulence, decompression, and medical emergencies.
        - Take into account the current weather conditions: {observations['origin_weather']['weather'][0]['description']} at the origin and {observations['destination_weather']['weather'][0]['description']} at the destination.

        ## Additional Information for Pilot and Crew:
        - Provide any relevant information for the pilot and crew, such as NOTAMs, airspace restrictions, oceanic procedures, and potential delays.
        """

        self.prompt = prompt  # Set the prompt attribute here

        response = await client.chat.completions.create(  # Use client initialized with AsyncOpenAI
            model="gpt-4o",  # or "gpt-3.5-turbo"
            messages=[{"role": "user", "content": prompt}],
        )
        flight_plan = response.choices[0].message.content.strip()

        # Post-processing (if needed)
        flight_plan = re.sub(r"\n+", "\n", flight_plan)
        flight_plan = re.sub(r"## (.*)", r"\n\n**\1**:", flight_plan)

        print('\n')
        print('Flight plan is running ....')
        print('\n\n')
        print("Flight Plan:", flight_plan)
        print('\n')

        return flight_plan


# Example usage
environment = FlightEnvironment(
    origin="New York",
    destination="London",
    weather={
        "New York": {"weather": [{"description": "Clear"}], "wind": {"speed": 5}},
        "London": {"weather": [{"description": "Overcast"}], "wind": {"speed": 10}},
    },
)


flight_agent = FlightAgent(environment, "/content/gdrive/MyDrive/datasets/turbulence/REF.csv")


async def main():
    """Executes the main logic of the flight agent."""
    # This is the correct way to execute the OODA loop as designed in FlightAgent.
    # It will perform observations, generate the flight plan, and simulate environment updates.
    await flight_agent.run(max_iterations=1)


import nest_asyncio

nest_asyncio.apply()  # Apply nest_asyncio to enable nested event loops

if __name__ == "__main__":
    print("Initializing Flight Planning Agent (gemini 2.0)")
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

Turbulence model trained successfully!
Initializing Flight Planning Agent (gemini 2.0)

--- OODA Loop Iteration 1 ---
Observations: {'origin_weather': {'weather': [{'description': 'Sunny'}], 'wind': {'speed': 5}}, 'destination_weather': {'weather': [{'description': 'Cloudy'}], 'wind': {'speed': 13}}, 'possible_routes': [{'flight_number': 'AC888', 'departure_time': '20:00', 'arrival_time': '08:00', 'price': 800, 'airline': 'Air Canada', 'aircraft_type': 'Boeing 787-9'}, {'flight_number': 'UA901', 'departure_time': '22:00', 'arrival_time': '10:00', 'price': 900, 'airline': 'United Airlines', 'aircraft_type': 'Boeing 777-300ER'}]}

            Current Situation:
            - Origin Weather: Sunny
            - Destination Weather: Cloudy
            - Available Routes: [{'flight_number': 'AC888', 'departure_time': '20:00', 'arrival_time': '08:00', 'price': 800, 'airline': 'Air Canada', 'aircraft_type': 'Boeing 787-9'}, {'flight_number': 'UA901', 'departure_time': '22:00', 'arrival_time':